In [14]:
#Load required libraries

'''Torch Libraries required to train Neural Network'''

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

'''Library required to train neural network using federated learning. Syft overloads PyTorch'''

import syft as sy

<h2>Setting of Learning Task</h2>

In [15]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.001
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 10
        self.save_model = False
        
args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

<h1>Load and Distribute Dataset</h1>

In [16]:
def Load_Data():
    
    '''Load CIFAR dataset from torch vision module and split to training/test set'''
    

    federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
    return (federated_train_loader,test_loader)

<h3>Initialize Workers</h3>

In [17]:
#Hook Torch instance with Syft Hook
hook = sy.TorchHook(torch)

#Initialize workers
me = hook.local_worker
bob = sy.VirtualWorker(id="bob",hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice",hook=hook, is_client_worker=False)

#Define compute nodes
compute_nodes = [bob, alice]

#Add worker instances
me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])


<h3>Distribute Data to Workers</h3>

In [ ]:
#TODO leverage FederatedDataset and FederatedDataloader

#Load required dataset
federated_train_loader,test_loader=Load_Data()

print(federated_train_loader.__len__())

#Distribute dataset among workers
#train_distributed_dataset = FederatedDataset(compute_nodes,trainloader)


Files already downloaded and verified
Scanning and sending data to bob, alice...


<h1>Define Neural Network Model</h1>

In [10]:
class Net(nn.Module):
    
    '''Neural Network Model'''
    
    def __init__(self):
        
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        
    def forward(self,x):
        
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,16*5*5)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return F.log_softmax(x, dim=1)    
    

<h1>Train Network</h1>

In [11]:

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    

In [12]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            criterion=nn.CrossEntropyLoss()
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "Federated_CIFAR10.pt")

/Users/hrishikesh/anaconda3/envs/syft_flstm/lib/python3.6/site-packages/syft-0.1.3a1-py3.6.egg/syft/frameworks/torch/tensors/interpreters/native.py:216: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  response = eval(cmd)(*args)


Train Epoch: 1 [0/50048 (0%)]	Loss: 2.301906
Train Epoch: 1 [640/50048 (1%)]	Loss: 2.305471
Train Epoch: 1 [1280/50048 (3%)]	Loss: 2.295641
Train Epoch: 1 [1920/50048 (4%)]	Loss: 2.309566
Train Epoch: 1 [2560/50048 (5%)]	Loss: 2.302849
Train Epoch: 1 [3200/50048 (6%)]	Loss: 2.316859
Train Epoch: 1 [3840/50048 (8%)]	Loss: 2.296539
Train Epoch: 1 [4480/50048 (9%)]	Loss: 2.311052
Train Epoch: 1 [5120/50048 (10%)]	Loss: 2.296306
Train Epoch: 1 [5760/50048 (12%)]	Loss: 2.310413
Train Epoch: 1 [6400/50048 (13%)]	Loss: 2.305417
Train Epoch: 1 [7040/50048 (14%)]	Loss: 2.300943
Train Epoch: 1 [7680/50048 (15%)]	Loss: 2.294203
Train Epoch: 1 [8320/50048 (17%)]	Loss: 2.292655
Train Epoch: 1 [8960/50048 (18%)]	Loss: 2.301416
Train Epoch: 1 [9600/50048 (19%)]	Loss: 2.296557
Train Epoch: 1 [10240/50048 (20%)]	Loss: 2.309853
Train Epoch: 1 [10880/50048 (22%)]	Loss: 2.309187
Train Epoch: 1 [11520/50048 (23%)]	Loss: 2.301913
Train Epoch: 1 [12160/50048 (24%)]	Loss: 2.301756
Train Epoch: 1 [12800/50048 

Train Epoch: 3 [3200/50048 (6%)]	Loss: 2.296962
Train Epoch: 3 [3840/50048 (8%)]	Loss: 2.306081
Train Epoch: 3 [4480/50048 (9%)]	Loss: 2.306893
Train Epoch: 3 [5120/50048 (10%)]	Loss: 2.304414
Train Epoch: 3 [5760/50048 (12%)]	Loss: 2.300484
Train Epoch: 3 [6400/50048 (13%)]	Loss: 2.303024
Train Epoch: 3 [7040/50048 (14%)]	Loss: 2.301856
Train Epoch: 3 [7680/50048 (15%)]	Loss: 2.308972
Train Epoch: 3 [8320/50048 (17%)]	Loss: 2.304061
Train Epoch: 3 [8960/50048 (18%)]	Loss: 2.301983
Train Epoch: 3 [9600/50048 (19%)]	Loss: 2.305756
Train Epoch: 3 [10240/50048 (20%)]	Loss: 2.307072
Train Epoch: 3 [10880/50048 (22%)]	Loss: 2.308706
Train Epoch: 3 [11520/50048 (23%)]	Loss: 2.292641
Train Epoch: 3 [12160/50048 (24%)]	Loss: 2.302902
Train Epoch: 3 [12800/50048 (26%)]	Loss: 2.291541
Train Epoch: 3 [13440/50048 (27%)]	Loss: 2.286159
Train Epoch: 3 [14080/50048 (28%)]	Loss: 2.302714
Train Epoch: 3 [14720/50048 (29%)]	Loss: 2.309542
Train Epoch: 3 [15360/50048 (31%)]	Loss: 2.301406
Train Epoch: 3

Train Epoch: 5 [6400/50048 (13%)]	Loss: 2.309510
Train Epoch: 5 [7040/50048 (14%)]	Loss: 2.300089
Train Epoch: 5 [7680/50048 (15%)]	Loss: 2.299743
Train Epoch: 5 [8320/50048 (17%)]	Loss: 2.299838
Train Epoch: 5 [8960/50048 (18%)]	Loss: 2.305449
Train Epoch: 5 [9600/50048 (19%)]	Loss: 2.301354
Train Epoch: 5 [10240/50048 (20%)]	Loss: 2.304584
Train Epoch: 5 [10880/50048 (22%)]	Loss: 2.302247
Train Epoch: 5 [11520/50048 (23%)]	Loss: 2.298271
Train Epoch: 5 [12160/50048 (24%)]	Loss: 2.300226
Train Epoch: 5 [12800/50048 (26%)]	Loss: 2.296985


KeyboardInterrupt: 